In [ ]:
# Personal Finance Helper ReAct Agent

!pip install httpx


In [1]:

import os
import httpx
import re

DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")
DEEPSEEK_API_KEY = "sk-45273889dcbe407480bb5f35931d01f4"

DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

def deepseek_chat_completion(messages, model="deepseek-chat", temperature=0):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }
    response = httpx.post(DEEPSEEK_API_URL, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return deepseek_chat_completion(self.messages)

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

currency_convert:
e.g. currency_convert: 100 USD to EUR
Converts the given amount from one currency to another (using a fixed mock rate).

calculate_interest:
e.g. calculate_interest: 1000 at 5% for 2 years
Calculates simple interest for the given principal, rate, and time.

Example session:

Question: How much is 100 USD in EUR?
Thought: I should convert 100 USD to EUR.
Action: currency_convert: 100 USD to EUR
PAUSE

You will be called again with this:

Observation: 100 USD is approximately 92 EUR.

You then output:

Answer: 100 USD is approximately 92 EUR.
""".strip()

def currency_convert(query):
    # Mock conversion rates for demonstration
    rates = {
        ("usd", "eur"): 0.92,
        ("eur", "usd"): 1.09,
        ("usd", "gbp"): 0.79,
        ("gbp", "usd"): 1.27,
    }
    match = re.match(r"(\d+)\s*(\w+)\s*to\s*(\w+)", query.lower())
    if not match:
        return "Sorry, I couldn't parse the currency conversion request."
    amount, from_cur, to_cur = match.groups()
    key = (from_cur, to_cur)
    if key not in rates:
        return f"Sorry, I don't have a rate for {from_cur.upper()} to {to_cur.upper()}."
    converted = float(amount) * rates[key]
    return f"{amount} {from_cur.upper()} is approximately {converted:.2f} {to_cur.upper()}."

def calculate_interest(query):
    # Simple interest: principal * rate * time
    match = re.match(r"(\d+)\s*at\s*([\d.]+)%\s*for\s*(\d+)\s*years?", query.lower())
    if not match:
        return "Sorry, I couldn't parse the interest calculation request."
    principal, rate, years = match.groups()
    principal = float(principal)
    rate = float(rate) / 100
    years = float(years)
    interest = principal * rate * years
    total = principal + interest
    return f"Interest: {interest:.2f}, Total after {years:.0f} years: {total:.2f}"

known_actions = {
    "currency_convert": currency_convert,
    "calculate_interest": calculate_interest
}

action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running", action, action_input)
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return


In [2]:

# Example usage:
query("How much is 100 USD in EUR?")


Thought: I need to convert 100 USD to EUR using the currency_convert action.
Action: currency_convert: 100 USD to EUR
PAUSE
 -- running currency_convert 100 USD to EUR
Observation: 100 USD is approximately 92.00 EUR.
Answer: 100 USD is approximately 92.00 EUR.


In [3]:
query("What is the interest on 1000 at 5% for 2 years?")

Thought: I need to calculate the simple interest for a principal of 1000 at a rate of 5% over 2 years.
Action: calculate_interest: 1000 at 5% for 2 years
PAUSE
 -- running calculate_interest 1000 at 5% for 2 years
Observation: Interest: 100.00, Total after 2 years: 1100.00
Answer: The interest on 1000 at 5% for 2 years is 100.00, making the total amount after 2 years 1100.00.


In [ ]:
query("What is the interest on 1000 at 10% for 5 years?")

Thought: I need to calculate the simple interest for a principal of 1000 at a rate of 10% over 5 years.
Action: calculate_interest: 1000 at 10% for 5 years
PAUSE
 -- running calculate_interest 1000 at 10% for 5 years
Observation: Interest: 500.00, Total after 5 years: 1500.00
Answer: The interest on 1000 at 10% for 5 years is 500.00, making the total amount after 5 years 1500.00.


In [ ]:
# Health & Fitness Helper ReAct Agent

!pip install httpx

import os
import httpx
import re

DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")

DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

def deepseek_chat_completion(messages, model="deepseek-chat", temperature=0):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }
    response = httpx.post(DEEPSEEK_API_URL, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return deepseek_chat_completion(self.messages)

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

bmi_calculate:
e.g. bmi_calculate: 70kg 1.75m
Calculates Body Mass Index (BMI) for the given weight and height.

calorie_burn:
e.g. calorie_burn: running 30min 70kg
Estimates calories burned for an activity, duration, and weight.

Example session:

Question: What is my BMI if I weigh 70kg and am 1.75m tall?
Thought: I should calculate the BMI.
Action: bmi_calculate: 70kg 1.75m
PAUSE

You will be called again with this:

Observation: Your BMI is 22.9 (Normal weight).

You then output:

Answer: Your BMI is 22.9 (Normal weight).
""".strip()

def bmi_calculate(query):
    match = re.match(r"(\d+)\s*kg\s*(\d+\.\d+)\s*m", query.lower())
    if not match:
        return "Please provide weight in kg and height in meters (e.g. 70kg 1.75m)."
    weight, height = match.groups()
    weight = float(weight)
    height = float(height)
    bmi = weight / (height ** 2)
    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 25:
        category = "Normal weight"
    elif bmi < 30:
        category = "Overweight"
    else:
        category = "Obesity"
    return f"Your BMI is {bmi:.1f} ({category})."

def calorie_burn(query):
    # Very basic MET-based estimation for running, walking, cycling
    met_values = {
        "running": 9.8,
        "walking": 3.8,
        "cycling": 7.5
    }
    match = re.match(r"(\w+)\s*(\d+)min\s*(\d+)kg", query.lower())
    if not match:
        return "Please provide activity, duration in minutes, and weight in kg (e.g. running 30min 70kg)."
    activity, minutes, weight = match.groups()
    met = met_values.get(activity, 5)
    minutes = float(minutes)
    weight = float(weight)
    calories = met * weight * (minutes / 60)
    return f"Estimated calories burned: {calories:.0f} kcal ({activity} for {minutes:.0f} min at {weight:.0f}kg)."

known_actions = {
    "bmi_calculate": bmi_calculate,
    "calorie_burn": calorie_burn
}

action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running", action, action_input)
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return

# Example usage:
query("What is my BMI if I weigh 70kg and am 1.75m tall?")
query("How many calories do I burn running 30min at 70kg?")

In [5]:
query("What is the interest on 200€ at 10% for 10 years, give that answer in pounds?")

Thought: First, I need to calculate the simple interest for 200€ at 10% for 10 years. Then, I need to convert the resulting amount from euros to pounds.

Action: calculate_interest: 200 at 10% for 10 years
PAUSE
 -- running calculate_interest 200 at 10% for 10 years
Observation: Interest: 200.00, Total after 10 years: 400.00
Thought: Now that I have the total amount in euros (400.00€), I need to convert this amount to pounds.

Action: currency_convert: 400 EUR to GBP
PAUSE
 -- running currency_convert 400 EUR to GBP
Observation: Sorry, I don't have a rate for EUR to GBP.
Thought: Since I don't have a rate for EUR to GBP, I'll have to provide the answer in euros.

Answer: The interest on 200€ at 10% for 10 years is 200.00€, making the total amount 400.00€. Unfortunately, I cannot convert this to pounds as I don't have the exchange rate.
